In [1]:
# Require libraries
import matplotlib.pyplot as plt
import os
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import numpy as np
import glob
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.signal import savgol_filter
from tsmoothie.smoother import LowessSmoother


In [2]:
# Function for pixel to degree conversion

# for stimulus
def px_deg_sti(data):
    time_sti = []
    pos_sti = []
    for y in data[1]:
        new_y = round((y - 960)/49.6,4) # horizontal center:960x540 pixel : 1 deg = 49.6 pixel
        pos_sti.append(new_y)

    for x in range(1,len(pos_sti)+1):
        new_x = round((1/60)*x,4) # for monitor with screen refresh rate( 60 Hz)
        time_sti.append(new_x)
    return time_sti,pos_sti
    
# for eye
def px_deg_eye(data,dominant_eye):
    time_eye = []
    pos_eye = []
    if dominant_eye == 'right':
        for y in data['GazePointRightX (ADCSpx)']:
            new_y = round((y - 960)/49.6,4) # horizontal center:960x540 pixel : 1 deg = 49.6 pixel
            pos_eye.append(new_y)
    elif dominant_eye == 'left':
        for y in data['GazePointLeftX (ADCSpx)']:
            new_y = round((y - 960)/49.6,4) # horizontal center:960x540 pixel : 1 deg = 49.6 pixel
            pos_eye.append(new_y)
    for x in range(1,len(pos_eye)+1):
        new_x = round((1/300)*x,4) # for tobii's eye tracker with 300 Hz sampling rate
        time_eye.append(new_x)
    return time_eye,pos_eye

In [3]:
# Create path for healthy control(pd)
path = os.getcwd()
dirname = os.path.dirname(path)
option = ['URLStart','URLEnd'] # This is the values in the Studio event column where stimuli start and end events from dataset
directory_pd  = dirname+"/eye_movements_sp/subjects/pd"
# s_path = dirname+"/result/4_dps/s_pd" # single gasf path for pd
# c_path = dirname+"/result/4_dps/c_pd" # combine gasf path for pd
files_pd = list(glob.iglob(directory_pd+'/[!.]*')) # skip unwanted file such as .DS_Store file
right = 'right'
left = 'left'

In [4]:
# For stimulus
directory_sti  = dirname+"/data/stimulus"
files_sti = glob.iglob(directory_sti+'/[!.]*')
for file_sti in files_sti: 
    # for stimulus (speed = 4deg/s, duration = 23 sec)
    data_sti = pd.read_csv(file_sti,sep='\s+',header=None)
    one_deg_time_sti , one_deg_pos_sti =  px_deg_sti(data_sti)
    stimulus = pd.DataFrame({'x':one_deg_time_sti,'y':one_deg_pos_sti})
    stimulus.to_csv(str(Path(file_sti).stem)+'_sti.csv',index=False)

## For PD

In [5]:
# one degree
def OneCsvPD(speed):
    frames = []
    for file_pd in files_pd:  
        print('File '+ str(os.path.basename(file_pd))+ ' is started.') 
        # Data preporcessing
        # Interpolate the NaN values with linear
        data_pd  = pd.read_excel(file_pd+'/'+speed)#.interpolate()
        t = data_pd[data_pd['StudioEvent'].isin(option)].index # Caculating start and end of stimuli
        
        if right in str(os.path.basename(file_pd)):
            # First Trial
            data_eye_pd_first = data_pd[t[0]+902:t[0]+27904] # first trial index
            #data_eye_pd_first  = remove_outlier_IQR(data_eye_pd_first,'GazePointRightX (ADCSpx)')
            #data_eye_pd_first  = data_eye_pd_first.to_frame()
            #data_eye_pd_first  =  blink_nan(data_eye_pd_first['GazePointRightX (ADCSpx)'],'GazePointRightX (ADCSpx)')
            #data_eye_pd_first  = data_eye_pd_first.to_frame()
            one_deg_time_eye , one_deg_pos_eye =  px_deg_eye(data_eye_pd_first,'right')
            
            #Remove left and right
            filename = os.path.basename(file_pd)
            filename = filename.replace('_right','').replace('_left','')
            one_deg_first = pd.DataFrame({'x2_'+str(filename):one_deg_time_eye,
            'y2_'+str(filename):one_deg_pos_eye})


            # Second Trial
            data_eye_pd_second = data_pd[t[2]+902:t[2]+27904]
            #data_eye_pd_second = remove_outlier_IQR(data_eye_pd_second,'GazePointRightX (ADCSpx)')
            #data_eye_pd_second  = data_eye_pd_second.to_frame()
            #data_eye_pd_second  =  blink_nan(data_eye_pd_second['GazePointRightX (ADCSpx)'],'GazePointRightX (ADCSpx)')
            #data_eye_pd_second  = data_eye_pd_second.to_frame()
            one_deg_time_eye_second , one_deg_pos_eye_second =  px_deg_eye(data_eye_pd_second,'right')
            
            #Remove left and right
            filename = os.path.basename(file_pd)
            filename = filename.replace('_right','').replace('_left','')
            one_deg_second = pd.DataFrame({'x2_'+str(filename):one_deg_time_eye_second,
            'y2_'+str(filename):one_deg_pos_eye_second})

            df_subject = pd.concat([one_deg_first,one_deg_second],axis=1)

            frames.append(df_subject)
            print('File '+ str(filename)+ ' is done.')
        else:
            # First Trial
            data_eye_pd_first = data_pd[t[0]+902:t[0]+27904] # first trial index
            #data_eye_pd_first  = remove_outlier_IQR(data_eye_pd_first,'GazePointLeftX (ADCSpx)')
            #data_eye_pd_first  = data_eye_pd_first.to_frame()
            #data_eye_pd_first  =  blink_nan(data_eye_pd_first['GazePointLeftX (ADCSpx)'],'GazePointLeftX (ADCSpx)')
            #data_eye_pd_first  = data_eye_pd_first.to_frame()
            one_deg_time_eye , one_deg_pos_eye =  px_deg_eye(data_eye_pd_first,'left')
            
            #Remove left and right
            filename = os.path.basename(file_pd)
            filename = filename.replace('_right','').replace('_left','')
            one_deg_first = pd.DataFrame({'x2_'+str(filename):one_deg_time_eye,
            'y2_'+str(filename):one_deg_pos_eye})


            # Second Trial
            data_eye_pd_second = data_pd[t[2]+902:t[2]+27904]
            #data_eye_pd_second = remove_outlier_IQR(data_eye_pd_second,'GazePointLeftX (ADCSpx)')
            #data_eye_pd_second  = data_eye_pd_second.to_frame()
            #data_eye_pd_second  =  blink_nan(data_eye_pd_second['GazePointLeftX (ADCSpx)'],'GazePointLeftX (ADCSpx)')
            #data_eye_pd_second  = data_eye_pd_second.to_frame()
            one_deg_time_eye_second , one_deg_pos_eye_second =  px_deg_eye(data_eye_pd_second,'left')
            
            #Remove left and right
            filename = os.path.basename(file_pd)
            filename = filename.replace('_right','').replace('_left','')
            one_deg_second = pd.DataFrame({'x2_'+str(filename):one_deg_time_eye_second,
            'y2_'+str(filename):one_deg_pos_eye_second})

            df_subject = pd.concat([one_deg_first,one_deg_second],axis=1)

            frames.append(df_subject)

            print('File '+ str(filename)+ ' is done.')


    one_deg = pd.concat(frames,axis=1)
   
    one_deg.to_csv('one_degPD.csv',index=False)

In [7]:
OneCsvPD('1_dps.xlsx')

In [8]:
# two degree
def TwoCsvPD(speed):
    frames = []
    for file_pd in files_pd:  
        # Data preporcessing
        # Interpolate the NaN values with linear
        data_pd  = pd.read_excel(file_pd+'/'+speed)#.interpolate()
        t = data_pd[data_pd['StudioEvent'].isin(option)].index # Caculating start and end of stimuli
        
        if right in str(os.path.basename(file_pd)):
            print('File '+ str(os.path.basename(file_pd))+ ' is started.') 
            # First Trial
            data_eye_pd_first = data_pd[t[0]+902:t[0]+14404] # first trial index
            #data_eye_pd_first  = remove_outlier_IQR(data_eye_pd_first,'GazePointRightX (ADCSpx)')
            #data_eye_pd_first  = data_eye_pd_first.to_frame()
            #data_eye_pd_first  =  blink_nan(data_eye_pd_first['GazePointRightX (ADCSpx)'],'GazePointRightX (ADCSpx)')
            #data_eye_pd_first  = data_eye_pd_first.to_frame()
            two_deg_time_eye , two_deg_pos_eye =  px_deg_eye(data_eye_pd_first,'right')
            
            #Remove left and right
            filename = os.path.basename(file_pd)
            filename = filename.replace('_right','').replace('_left','')
            
            #Remove left and right
            filename = os.path.basename(file_pd)
            filename = filename.replace('_right','').replace('_left','')
            two_deg_first = pd.DataFrame({'x2_'+str(filename):two_deg_time_eye,
            'y2_'+str(filename):two_deg_pos_eye})


            # Second Trial
            data_eye_pd_second = data_pd[t[2]+902:t[2]+14404]
            #data_eye_pd_second= remove_outlier_IQR(data_eye_pd_second,'GazePointRightX (ADCSpx)')
            #data_eye_pd_second  = data_eye_pd_second.to_frame()
            #data_eye_pd_second  =  blink_nan(data_eye_pd_second['GazePointRightX (ADCSpx)'],'GazePointRightX (ADCSpx)')
            #data_eye_pd_second  = data_eye_pd_second.to_frame()
            two_deg_time_eye_second , two_deg_pos_eye_second =  px_deg_eye(data_eye_pd_second,'right')
            
            #Remove left and right
            filename = os.path.basename(file_pd)
            filename = filename.replace('_right','').replace('_left','')
            two_deg_second = pd.DataFrame({'x2_'+str(filename):two_deg_time_eye_second,
            'y2_'+str(filename):two_deg_pos_eye_second})

            df_subject = pd.concat([two_deg_first,two_deg_second],axis=1)

            frames.append(df_subject)

            print('File '+ str(filename)+ ' is done.')
        else:
            # First Trial
            data_eye_pd_first = data_pd[t[0]+902:t[0]+14404] # first trial index
            #data_eye_pd_first = remove_outlier_IQR(data_eye_pd_first,'GazePointLeftX (ADCSpx)')
            #data_eye_pd_first  = data_eye_pd_first.to_frame()
            #data_eye_pd_first  =  blink_nan(data_eye_pd_first['GazePointLeftX (ADCSpx)'],'GazePointLeftX (ADCSpx)')
            #data_eye_pd_first  = data_eye_pd_first.to_frame()
            two_deg_time_eye , two_deg_pos_eye =  px_deg_eye(data_eye_pd_first,'left')
            
            #Remove left and right
            filename = os.path.basename(file_pd)
            filename = filename.replace('_right','').replace('_left','')
            two_deg_first = pd.DataFrame({'x2_'+str(filename):two_deg_time_eye,
            'y2_'+str(filename):two_deg_pos_eye})


            # Second Trial
            data_eye_pd_second = data_pd[t[2]+902:t[2]+14404]
            #data_eye_pd_second = remove_outlier_IQR(data_eye_pd_second,'GazePointLeftX (ADCSpx)')
            #data_eye_pd_second  = data_eye_pd_second.to_frame()
            #data_eye_pd_second  =  blink_nan(data_eye_pd_second['GazePointLeftX (ADCSpx)'],'GazePointLeftX (ADCSpx)')
            #data_eye_pd_second  = data_eye_pd_second.to_frame()
            two_deg_time_eye_second , two_deg_pos_eye_second =  px_deg_eye(data_eye_pd_second,'left')

            #Remove left and right
            filename = os.path.basename(file_pd)
            filename = filename.replace('_right','').replace('_left','')
            two_deg_second = pd.DataFrame({'x2_'+str(filename):two_deg_time_eye_second,
            'y2_'+str(filename):two_deg_pos_eye_second})

            df_subject = pd.concat([two_deg_first,two_deg_second],axis=1)

            frames.append(df_subject)

            print('File '+ str(filename)+ ' is done.')


    two_deg = pd.concat(frames,axis=1)
   
    two_deg.to_csv('two_degPD.csv',index=False)

In [10]:
TwoCsvPD('2_dps.xlsx')

In [11]:
# Four degree
def FourCsvPD(speed):
    frames = []
    for file_pd in files_pd:  
        print('File '+ str(os.path.basename(file_pd))+ ' is started.') 
        # Data preporcessing
        # Interpolate the NaN values with linear
        data_pd  = pd.read_excel(file_pd+'/'+speed)#.interpolate()
        t = data_pd[data_pd['StudioEvent'].isin(option)].index # Caculating start and end of stimuli
        
        if right in str(os.path.basename(file_pd)):
            # First Trial
            data_eye_pd_first = data_pd[t[0]+902:t[0]+7804] # first trial index
            #data_eye_pd_first  = remove_outlier_IQR(data_eye_pd_first,'GazePointRightX (ADCSpx)')
            #outlier_first  = outlier_first.to_frame()
            #data_eye_pd_first  =  blink_nan(data_eye_pd_first['GazePointRightX (ADCSpx)'],'GazePointRightX (ADCSpx)')
            #data_eye_pd_first  = data_eye_pd_first.to_frame()
            four_deg_time_eye , four_deg_pos_eye =  px_deg_eye(data_eye_pd_first,'right')
            
            #Remove left and right
            filename = os.path.basename(file_pd)
            filename = filename.replace('_right','').replace('_left','')
            four_deg_first = pd.DataFrame({'x2_'+str(filename):four_deg_time_eye,
            'y2_'+str(filename):four_deg_pos_eye})


            # Second Trial
            data_eye_pd_second = data_pd[t[2]+902:t[2]+7804]
            #data_eye_pd_second = remove_outlier_IQR(data_eye_pd_second,'GazePointRightX (ADCSpx)')
            #data_eye_pd_second  = data_eye_pd_second.to_frame()
            #data_eye_pd_second  =  blink_nan(data_eye_pd_second['GazePointRightX (ADCSpx)'],'GazePointRightX (ADCSpx)')
            #data_eye_pd_second  = data_eye_pd_second.to_frame()
            four_deg_time_eye_second , four_deg_pos_eye_second =  px_deg_eye(data_eye_pd_second,'right')
            
            #Remove left and right
            filename = os.path.basename(file_pd)
            filename = filename.replace('_right','').replace('_left','')
            four_deg_second = pd.DataFrame({'x2_'+str(filename):four_deg_time_eye_second,
            'y2_'+str(filename):four_deg_pos_eye_second})

            df_subject = pd.concat([four_deg_first,four_deg_second],axis=1)

            frames.append(df_subject)

            print('File '+ str(filename)+ ' is done.')
        else:
            # First Trial
            data_eye_pd_first = data_pd[t[0]+902:t[0]+7804] # first trial index
            #data_eye_pd_first  = remove_outlier_IQR(data_eye_pd_first,'GazePointLeftX (ADCSpx)')
            #data_eye_pd_first  = data_eye_pd_first.to_frame()
            #data_eye_pd_first  =  blink_nan(data_eye_pd_first['GazePointLeftX (ADCSpx)'],'GazePointLeftX (ADCSpx)')
            #data_eye_pd_first  = data_eye_pd_first.to_frame()
            four_deg_time_eye , four_deg_pos_eye =  px_deg_eye(data_eye_pd_first,'left')

            #Remove left and right
            filename = os.path.basename(file_pd)
            filename = filename.replace('_right','').replace('_left','')
            four_deg_first = pd.DataFrame({'x2_'+str(filename):four_deg_time_eye,
            'y2_'+str(filename):four_deg_pos_eye})


            # Second Trial
            data_eye_pd_second = data_pd[t[2]+902:t[2]+7804]
            #print("Before remove_outlier_IQR:", data_eye_pd_second.columns)
            #data_eye_pd_second = remove_outlier_IQR(data_eye_pd_second,'GazePointLeftX (ADCSpx)')
            #data_eye_pd_second  = data_eye_pd_second.to_frame()
             # Add this line to print the columns
            #print("After remove_outlier_IQR:", data_eye_pd_second.name)
            #data_eye_pd_second  =  blink_nan(data_eye_pd_second['GazePointLeftX (ADCSpx)'],'GazePointLeftX (ADCSpx)')
            #data_eye_pd_second  = data_eye_pd_second.to_frame()
            four_deg_time_eye_second , four_deg_pos_eye_second =  px_deg_eye(data_eye_pd_second,'left')
            
            #Remove left and right
            filename = os.path.basename(file_pd)
            filename = filename.replace('_right','').replace('_left','')
            four_deg_second = pd.DataFrame({'x2_'+str(filename):four_deg_time_eye_second,
            'y2_'+str(filename):four_deg_pos_eye_second})

            df_subject = pd.concat([four_deg_first,four_deg_second],axis=1)

            frames.append(df_subject)

            print('File '+ str(filename)+ ' is done.')


    four_deg = pd.concat(frames,axis=1)
   
    four_deg.to_csv('four_degPD.csv',index=False)

In [15]:
FourCsvPD('4_dps.xlsx')

In [16]:
# six degree
def SixCsvPD(speed):
    frames = []
    for file_pd in files_pd:  
        # Data preporcessing
        # Interpolate the NaN values with linear
        data_pd  = pd.read_excel(file_pd+'/'+speed)#.interpolate()
        t = data_pd[data_pd['StudioEvent'].isin(option)].index # Caculating start and end of stimuli
        
        if right in str(os.path.basename(file_pd)):
            print('File '+ str(os.path.basename(file_pd))+ ' is started.') 
            # First Trial
            data_eye_pd_first = data_pd[t[0]+902:t[0]+5404] # first trial index
            #data_eye_pd_first  = remove_outlier_IQR(data_eye_pd_first,'GazePointRightX (ADCSpx)')
            #data_eye_pd_first  = data_eye_pd_first.to_frame()
            #data_eye_pd_first  =  blink_nan(data_eye_pd_first['GazePointRightX (ADCSpx)'],'GazePointRightX (ADCSpx)')
            #data_eye_pd_first  = data_eye_pd_first.to_frame()
            six_deg_time_eye , six_deg_pos_eye =  px_deg_eye(data_eye_pd_first,'right')
            
            #Remove left and right
            filename = os.path.basename(file_pd)
            filename = filename.replace('_right','').replace('_left','')
            six_deg_first = pd.DataFrame({'x2_'+str(filename):six_deg_time_eye,
            'y2_'+str(filename):six_deg_pos_eye})



            # Second Trial
            data_eye_pd_second = data_pd[t[2]+902:t[2]+5404]
            #data_eye_pd_second = remove_outlier_IQR(data_eye_pd_second,'GazePointRightX (ADCSpx)')
            #data_eye_pd_second  = data_eye_pd_second.to_frame()
            #data_eye_pd_second  =  blink_nan(data_eye_pd_second['GazePointRightX (ADCSpx)'],'GazePointRightX (ADCSpx)')
            #data_eye_pd_second  = data_eye_pd_second.to_frame()
            six_deg_time_eye_second , six_deg_pos_eye_second =  px_deg_eye(data_eye_pd_second,'right')
            
            #Remove left and right
            filename = os.path.basename(file_pd)
            filename = filename.replace('_right','').replace('_left','')
            six_deg_second = pd.DataFrame({'x2_'+str(filename):six_deg_time_eye_second,
            'y2_'+str(filename):six_deg_pos_eye_second})

            df_subject = pd.concat([six_deg_first,six_deg_second],axis=1)

            frames.append(df_subject)

            print('File '+ str(filename)+ ' is done.')
        else:
            # First Trial
            data_eye_pd_first = data_pd[t[0]+902:t[0]+5404] # first trial index
            #data_eye_pd_first  = remove_outlier_IQR(data_eye_pd_first,'GazePointLeftX (ADCSpx)')
            #data_eye_pd_first  = data_eye_pd_first.to_frame()
            #data_eye_pd_first  =  blink_nan(data_eye_pd_first['GazePointLeftX (ADCSpx)'],'GazePointLeftX (ADCSpx)')
            #data_eye_pd_first  = data_eye_pd_first.to_frame()
            six_deg_time_eye , six_deg_pos_eye =  px_deg_eye(data_eye_pd_first,'left')
            
            #Remove left and right
            filename = os.path.basename(file_pd)
            filename = filename.replace('_right','').replace('_left','')
            six_deg_first = pd.DataFrame({'x2_'+str(filename):six_deg_time_eye,
            'y2_'+str(filename):six_deg_pos_eye})



            # Second Trial
            data_eye_pd_second = data_pd[t[2]+902:t[2]+5404]
            #data_eye_pd_second = remove_outlier_IQR(data_eye_pd_second,'GazePointLeftX (ADCSpx)')
            #data_eye_pd_second  = data_eye_pd_second.to_frame()
            #data_eye_pd_second  =  blink_nan(data_eye_pd_second['GazePointLeftX (ADCSpx)'],'GazePointLeftX (ADCSpx)')
            #data_eye_pd_second  = data_eye_pd_second.to_frame()
            six_deg_time_eye_second , six_deg_pos_eye_second =  px_deg_eye(data_eye_pd_second,'left')

            #Remove left and right
            filename = os.path.basename(file_pd)
            filename = filename.replace('_right','').replace('_left','')
            six_deg_second = pd.DataFrame({'x2_'+str(filename):six_deg_time_eye_second,
            'y2_'+str(filename):six_deg_pos_eye_second})

            df_subject = pd.concat([six_deg_first,six_deg_second],axis=1)

            frames.append(df_subject)

            print('File '+ str(filename)+ ' is done.')


    six_deg = pd.concat(frames,axis=1)
   
    six_deg.to_csv('six_degPD.csv',index=False)

In [18]:
SixCsvPD('6_dps.xlsx')

In [19]:
# eight degree
def EightCsvPD(speed):
    frames = []
    for file_pd in files_pd:  
        # Data preporcessing
        # Interpolate the NaN values with linear
        data_pd  = pd.read_excel(file_pd+'/'+speed)#.interpolate()
        t = data_pd[data_pd['StudioEvent'].isin(option)].index # Caculating start and end of stimuli
        
        if right in str(os.path.basename(file_pd)):
            print('File '+ str(os.path.basename(file_pd))+ ' is started.') 
            # First Trial
            data_eye_pd_first = data_pd[t[0]+902:t[0]+4204] # first trial index
            #data_eye_pd_first  = remove_outlier_IQR(data_eye_pd_first,'GazePointRightX (ADCSpx)')
            #data_eye_pd_first  = data_eye_pd_first.to_frame()
            #data_eye_pd_first  =  blink_nan(data_eye_pd_first['GazePointRightX (ADCSpx)'],'GazePointRightX (ADCSpx)')
            #data_eye_pd_first  = data_eye_pd_first.to_frame()
            eight_deg_time_eye , eight_deg_pos_eye =  px_deg_eye(data_eye_pd_first,'right')

             #Remove left and right
            filename = os.path.basename(file_pd)
            filename = filename.replace('_right','').replace('_left','')
            eight_deg_first = pd.DataFrame({'x2_'+str(filename):eight_deg_time_eye,
            'y2_'+str(filename):eight_deg_pos_eye})

            # Second Trial
            data_eye_pd_second = data_pd[t[2]+902:t[2]+4204]
            #data_eye_pd_second = remove_outlier_IQR(data_eye_pd_second,'GazePointRightX (ADCSpx)')
            #data_eye_pd_second  = data_eye_pd_second.to_frame()
            #data_eye_pd_second  =  blink_nan(data_eye_pd_second['GazePointRightX (ADCSpx)'],'GazePointRightX (ADCSpx)')
            #data_eye_pd_second  = data_eye_pd_second.to_frame()
            eight_deg_time_eye_second , eight_deg_pos_eye_second =  px_deg_eye(data_eye_pd_second,'right')
            
            #Remove left and right
            filename = os.path.basename(file_pd)
            filename = filename.replace('_right','').replace('_left','')
            eight_deg_second = pd.DataFrame({'x2_'+str(filename):eight_deg_time_eye_second,
            'y2_'+str(filename):eight_deg_pos_eye_second})


            df_subject = pd.concat([eight_deg_first,eight_deg_second],axis=1)

            frames.append(df_subject)

            print('File '+ str(filename)+ ' is done.')
        else:
            # First Trial
            data_eye_pd_first = data_pd[t[0]+902:t[0]+4204] # first trial index
            #data_eye_pd_first  = remove_outlier_IQR(data_eye_pd_first,'GazePointLeftX (ADCSpx)')
            #data_eye_pd_first  = data_eye_pd_first.to_frame()
            #data_eye_pd_first  =  blink_nan(data_eye_pd_first['GazePointLeftX (ADCSpx)'],'GazePointLeftX (ADCSpx)')
            #data_eye_pd_first  = data_eye_pd_first.to_frame()
            eight_deg_time_eye , eight_deg_pos_eye =  px_deg_eye(data_eye_pd_first,'left')

            #Remove left and right
            filename = os.path.basename(file_pd)
            filename = filename.replace('_right','').replace('_left','')
            eight_deg_first = pd.DataFrame({'x2_'+str(filename):eight_deg_time_eye,
            'y2_'+str(filename):eight_deg_pos_eye})


            # Second Trial
            data_eye_pd_second = data_pd[t[2]+902:t[2]+4204]
            #data_eye_pd_second= remove_outlier_IQR(data_eye_pd_second,'GazePointLeftX (ADCSpx)')
            #data_eye_pd_second  = data_eye_pd_second.to_frame()
            #data_eye_pd_second  =  blink_nan(data_eye_pd_second['GazePointLeftX (ADCSpx)'],'GazePointLeftX (ADCSpx)')
            #data_eye_pd_second  = data_eye_pd_second.to_frame()
            eight_deg_time_eye_second , eight_deg_pos_eye_second =  px_deg_eye(data_eye_pd_second,'left')

            #Remove left and right
            filename = os.path.basename(file_pd)
            filename = filename.replace('_right','').replace('_left','')
            eight_deg_second = pd.DataFrame({'x2_'+str(filename):eight_deg_time_eye_second,
            'y2_'+str(filename):eight_deg_pos_eye_second})
            

            df_subject = pd.concat([eight_deg_first,eight_deg_second],axis=1)

            frames.append(df_subject)
            print('File '+ str(filename)+ ' is done.')


    eight_deg = pd.concat(frames,axis=1)
   
    eight_deg.to_csv('eight_degPD.csv',index=False)

In [21]:
EightCsvPD('8_dps.xlsx')